# Finding latent dimensions for the toroidal mirror 

It is common that beamline inputs are highly coupled, and so the effect of an input on the beam cannot be understood except in concert with the others. In this example, we show how our agent figures out latent dimensions, as well as the benefit of doing so. 

In [ ]:
%run -i ~/repos/bloptools/examples/prepare_bluesky.py
%run -i ~/repos/bloptools/examples/prepare_tes_shadow.py

toroid_dofs = np.array([toroid.x_rot, toroid.offz])
toroid_bounds = np.array([[-1e-3, 1e-3], [-4e-1, +4e-1]])

In [ ]:
from bloptools.bayesian import Agent
from bloptools.experiments.sirepo.tes import w8_digestion
from bloptools.tasks import Task

beam_flux_task = Task(key="flux", kind="max", transform=lambda x: np.log(x))

agent = Agent(
    active_dofs=toroid_dofs,
    active_dof_bounds=toroid_bounds,
    detectors=[w8],
    tasks=[beam_flux_task],
    digestion=w8_digestion,
    db=db,
)

RE(agent.initialize(acqf="qr", n_init=24))

We can see that the beam is only not cut off (i.e. it has a non-zero flux) in a diagonal strip, and that in fact this is really just a one-dimensional optimization problem in some diagonal dimension. Our agent has figured this out, with a transformation matrix that has a long coherence length in one dimension and a short coherence length orthogonal to it:

In [ ]:
agent.tasks[0].regressor.covar_module.latent_dimensions

In [ ]:
agent.plot_tasks()
agent.plot_feasibility()
agent.plot_acquisition(strategy=["ei", "pi", "ucb"])